# Работа с данными. Домашнее задание


## Задача
Имеется набор данных покупателей супермаркета. Проведите анализ и очистку этих данных.

### Описание данных

Дан файл Mall_Customers.csv, содержащий следующие данные по покупателям сети супермаркетов:
- CustomerID — идентификатор покупателя.
- Genre – пол покупателя.
- Age – возраст покупателя.
- Annual Income (k\$) – годовой доход покупателя, тысяч $.
- Spending Score (1–100) – рейтинг покупателя, целевая переменная.

### Задание 1

Загрузите данные из файла `Mall_Customers.csv` в ноутбук и выведите первые пять строк на экран.

In [1]:
### YOUR CODE HERE ###
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Mall_Customers.csv')
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19.0,15.0,39
1,2,Male,NaN,NaN,81
2,3,Female,NaN,16.0,6
3,4,Female,23.0,16.0,77
4,5,Female,31.0,17.0,40


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CustomerID              200 non-null    int64  
 1   Genre                   181 non-null    object 
 2   Age                     180 non-null    float64
 3   Annual Income (k$)      180 non-null    float64
 4   Spending Score (1-100)  200 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 7.9+ KB


### Задание 2

Проведите разведочный анализ загруженных данных. Обязательные атрибуты для анализа:
- количество пропусков в каждом признаке;
- распределения признаков;
- возможные выбросы или аномалии.

Анализ дополнительных атрибутов и свойств данных приветствуется. Используйте средства визуализации там, где это необходимо.

По результатам анализа сделайте выводы о свойствах отдельных признаков и качестве данных в целом.

In [12]:
### YOUR CODE HERE ###
df.isna().sum()

CustomerID                 0
Genre                     19
Age                       20
Annual Income (k$)        20
Spending Score (1-100)     0
dtype: int64

In [19]:
# Заполнение пропусков

# Genre → мода
df["Genre"] = df["Genre"].fillna(df["Genre"].mode()[0])

# Age → медиана
df["Age"]  = df["Age"].fillna(df["Age"].median())

# Income → медиана
df["Annual Income (k$)"]  = df["Annual Income (k$)"].fillna(df["Annual Income (k$)"].median())


In [20]:
df.isna().sum()

CustomerID                0
Genre                     0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

### Задание 3

Разделите данные на обучающую и тестовую выборки в пропорции 80:20. Здесь и далее используйте random_state = 1.

In [21]:
### YOUR CODE HERE ###
from sklearn.model_selection import train_test_split

X = df.drop("CustomerID", axis=1)   # CustomerID не нужен как признак
y = None  # В этой задаче нет целевой переменной — это препроцессинг

X_train, X_test = train_test_split(
    X, test_size=0.2, random_state=1
)

print(X_train.shape, X_test.shape)

(160, 4) (40, 4)


### Задание 4

Заполните пропуски в признаках в обучающей выборке с использованием инструментов sklearn. Для каждого признака выберите и аргументируйте стратегию заполнения пропусков.

In [22]:
### YOUR CODE HERE ###
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

num_features = ["Age", "Annual Income (k$)"]
cat_features = ["Genre"]

imputer = ColumnTransformer(
    transformers=[
        ("num_imputer", SimpleImputer(strategy="median"), num_features),
        ("cat_imputer", SimpleImputer(strategy="most_frequent"), cat_features)
    ],
    remainder="passthrough"
)

X_train_imp = imputer.fit_transform(X_train)


### Задание 5

Найдите выбросы в признаках обучающей выборки. Замените выбросы, если таковые имеются, и аргументируйте выбор метода поиска выбросов.


In [23]:
### YOUR CODE HERE ###

X_train_imp_df = pd.DataFrame(
    X_train_imp,
    columns=num_features + cat_features + ["Spending Score (1-100)"]
)

# Ищем выбросы методом IQR только для числовых
for col in num_features + ["Spending Score (1-100)"]:
    Q1 = X_train_imp_df[col].quantile(0.25)
    Q3 = X_train_imp_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Замена выбросов на границы
    X_train_imp_df[col] = np.where(X_train_imp_df[col] < lower, lower,
                                   np.where(X_train_imp_df[col] > upper, upper, X_train_imp_df[col]))


### Задание 6

Нормализуйте численные признаки. Аргументируйте выбор стратегии нормализации для каждого признака.


In [24]:
### YOUR CODE HERE ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler

standard_cols = ["Age"]
minmax_cols = ["Annual Income (k$)", "Spending Score (1-100)"]

scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()

X_train_imp_df[standard_cols] = scaler_std.fit_transform(X_train_imp_df[standard_cols])
X_train_imp_df[minmax_cols] = scaler_mm.fit_transform(X_train_imp_df[minmax_cols])


### Задание 7

Закодируйте категориальные признаки в обучающей выборке. Аргументируйте выбор способа кодирования.

In [27]:
### YOUR CODE HERE ###
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop="first", sparse_output=False)

encoded = encoder.fit_transform(X_train_imp_df[["Genre"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(["Genre"]))

# объединяем
X_train_final = pd.concat([X_train_imp_df.drop("Genre", axis=1).reset_index(drop=True),
                           encoded_df], axis=1)

### Задание 8
Проведите очистку и подготовку тестовых данных. Используйте препроцессоры и другие инструменты, которые вы уже использовали при подготовке обучающей выборки, без их дополнительной настройки.

In [30]:
# 1. Импутация
X_test_imp = imputer.transform(X_test)

X_test_imp_df = pd.DataFrame(
    X_test_imp,
    columns=num_features + cat_features + ["Spending Score (1-100)"]
)

# 2. Winsorization выбросов теми же границами — но мы применили замену на границы через transform, значит на Test тоже:
for col in num_features + ["Spending Score (1-100)"]:
    Q1 = X_train_imp_df[col].quantile(0.25)
    Q3 = X_train_imp_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    X_test_imp_df[col] = np.where(X_test_imp_df[col] < lower, lower,
                                  np.where(X_test_imp_df[col] > upper, upper, X_test_imp_df[col]))

# 3. Нормализация (transform!)
X_test_imp_df[standard_cols] = scaler_std.transform(X_test_imp_df[standard_cols])
X_test_imp_df[minmax_cols] = scaler_mm.transform(X_test_imp_df[minmax_cols])

# 4. OHE-кодирование
encoded_test = encoder.transform(X_test_imp_df[["Genre"]])
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(["Genre"]))

X_test_final = pd.concat(
    [X_test_imp_df.drop("Genre", axis=1).reset_index(drop=True), encoded_test_df],
    axis=1
)
